In [1]:
import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt

import os

import glob
import tqdm
import pandas as pd
from natsort import natsorted
from os.path import basename
import numpy as np
import pickle

from sklearn.svm import SVR
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import GradientBoostingRegressor

from model import *
from mining import *



Using TensorFlow backend.


In [3]:
y_arousal = np.load('./DEAM/y_arousal_mean.npy')
y_valence = np.load('./DEAM/y_valence_mean.npy')
X = np.load('./DEAM/X_DEAM.npy')
X = np.average(X,axis=1)

In [4]:
adam_optim = Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, decay=0.0, amsgrad=False)

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.2
set_session(tf.Session(config=config))        
    
input_dim = X.shape[1]
anchor_input = Input((input_dim, ), name='anchor_input')
positive_input = Input((input_dim, ), name='positive_input')
negative_input = Input((input_dim, ), name='negative_input')

# Shared embedding layer for positive and negative items
Shared_DNN = create_base_network(input_dim, 260)


encoded_anchor = Shared_DNN(anchor_input)
encoded_positive = Shared_DNN(positive_input)
encoded_negative = Shared_DNN(negative_input)

merged_vector = concatenate([encoded_anchor, encoded_positive, encoded_negative], axis=-1)
model = Model(inputs=[anchor_input,positive_input, negative_input], outputs=merged_vector)

X_scaler = StandardScaler()
arousal_scaler = MinMaxScaler((-1,1))
valence_scaler = MinMaxScaler((-1,1))

svm_r2_history = []
svm_mse_history = []

total_fold = 10
kf = KFold(n_splits=total_fold, random_state=101)
fold = 0
for i in kf.split(X,y_arousal):
    print('Now working on {} Fold'.format(fold))
    train_index = i[0]
    test_index = i[1]
    
    X_train_norm = X_scaler.fit_transform(X[train_index])
    X_test_norm = X_scaler.transform(X[test_index])
    
    y_a_train_norm = arousal_scaler.fit_transform(y_arousal[train_index].reshape(-1,1))
    y_a_test_norm = arousal_scaler.transform(y_arousal[test_index].reshape(-1,1))
    
    y_v_train_norm = valence_scaler.fit_transform(y_valence[train_index].reshape(-1,1))
    y_v_test_norm = valence_scaler.transform(y_valence[test_index].reshape(-1,1))
    
    y_train_R_norm = y_v_train_norm # This model is training on valence, to train on arousal, just change this line
    y_test_R_norm = y_v_test_norm

    model.compile(loss=triplet_loss, optimizer=adam_optim)

    model.summary()

    total_loss_history = []
    total_valloss_history = []
    r2_history = []
    baseline_r2_history = []
    mse_history = []
    baseline_mse_history =[]
    for i in range(25): # Mining triplets for 25 times
        if i == 0:
            r2_old=0
            r2 = 0
        else:
            # if r2 > r2_old:
            # model.save_weights(foldername + filename + '.hdf5')
            # else:
            pass
                
        
        
        if r2 > r2_old:
            print('saving...........')
            r2_old = r2 #saving old r2 for comparison
            set_folder = './TNN_training/'
            filename = 'GBM_valence_{}_of_{}fold_triplet_260_'.format(fold, total_fold) + str(i) + 'nd_r2-' + str(r2)
            try:
                foldername = set_folder
                filename=filename
                model.save_weights(foldername+filename+'.hdf5')
            except:
                os.mkdir(set_folder)
                filename=filename
                model.save_weights(foldername+filename+'.hdf5')                
        
        triplet_train, triplet_test = generate_triplet_R(X_train_norm,y_train_R_norm,positive_range=0.1, negative_range=0.5,  ap_pairs=10,an_pairs=10)
        Anchor = triplet_train[:,0,:]
        Positive = triplet_train[:,1,:]
        Negative = triplet_train[:,2,:]
        Anchor_test = triplet_test[:,0,:]
        Positive_test = triplet_test[:,1,:]
        Negative_test = triplet_test[:,2,:]
        Y_dummy = np.empty((Anchor.shape[0],1))
        Y_dummy2 = np.empty((Anchor_test.shape[0],1))


        history = model.fit([Anchor,Positive,Negative],y=Y_dummy,validation_data=([Anchor_test,Positive_test,Negative_test],Y_dummy2), batch_size=128, epochs=10,verbose=True)
        total_loss_history = total_loss_history + history.history['loss']
        total_valloss_history = total_valloss_history + history.history['val_loss']

        trained_model = Model(inputs=anchor_input, outputs=encoded_anchor)
        transformed_X_train = trained_model.predict(X_train_norm)
        transformed_X_test = trained_model.predict(X_test_norm)

        regressor_model = GradientBoostingRegressor()
        regressor_model.fit(transformed_X_train, y_train_R_norm)

        gradboost_y_pred_norm = regressor_model.predict(transformed_X_test)
        # gradboost_y_pred = scaler_yv.inverse_transform(gradboost_y_pred_norm.reshape(-1,1))

        mse = mean_squared_error(y_test_R_norm,gradboost_y_pred_norm)
        r2 = r2_score(y_test_R_norm, gradboost_y_pred_norm)
        print('mse = {}'.format(mse))
        print('r2 = {}'.format(r2))   
        r2_history.append(r2)
        mse_history.append(mse)


    fig = plt.figure(figsize=[15,5])
    ax1 = fig.add_subplot(121)
    ax1.set_xlabel('epochs')
    ax1.set_ylabel('loss')
    ax1.plot(total_loss_history)
    ax1.plot(total_valloss_history)
    ax1.legend(['loss','val_loss'])

    ax2 = fig.add_subplot(122)
    ax2.set_xlabel('epochs')
    ax2.set_ylabel('r2')
    ax2.plot(r2_history)
    ax2.plot(baseline_r2_history)
    ax2.legend(['r2'])
    pickle.dump(r2_history, open('GBM_valence_{}_of_{}fold_r2'.format(fold,total_fold), 'wb'))
    pickle.dump(mse_history,open('GBM_valence_{}_of_{}fold_mse'.format(fold,total_fold), 'wb'))
    fig.savefig('GBM_valence_training_loss_{}_of_{}fold'.format(fold,total_fold))
    fold+=1       
    
    break # for demonstration, we just try on 1 fold

Now working on 0 Fold
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
anchor_input (InputLayer)       (None, 260)          0                                            
__________________________________________________________________________________________________
positive_input (InputLayer)     (None, 260)          0                                            
__________________________________________________________________________________________________
negative_input (InputLayer)     (None, 260)          0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 260)          67860       anchor_input[0][0]               
                                                                 posit

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


mse = 0.1201624600617371
r2 = 0.06750834924327187
saving...........
Train on 112070 samples, validate on 48080 samples
Epoch 1/10
112070/112070 [==============================] - 3s 30us/step - loss: 3.1184 - val_loss: 3.0068
Epoch 2/10
112070/112070 [==============================] - 4s 34us/step - loss: 2.7363 - val_loss: 2.7247
Epoch 3/10
112070/112070 [==============================] - 3s 30us/step - loss: 2.4641 - val_loss: 2.5126
Epoch 4/10
112070/112070 [==============================] - 3s 30us/step - loss: 2.2515 - val_loss: 2.3419
Epoch 5/10
112070/112070 [==============================] - 3s 31us/step - loss: 2.0792 - val_loss: 2.2022
Epoch 6/10
112070/112070 [==============================] - 3s 31us/step - loss: 1.9368 - val_loss: 2.0854
Epoch 7/10
112070/112070 [==============================] - 3s 29us/step - loss: 1.8157 - val_loss: 1.9853
Epoch 8/10
112070/112070 [==============================] - 3s 31us/step - loss: 1.7120 - val_loss: 1.8990
Epoch 9/10
112070/112070 

112070/112070 [==============================] - 4s 36us/step - loss: 0.6083 - val_loss: 0.7326
mse = 0.10436550462357051
r2 = 0.19009679363678855
Train on 112070 samples, validate on 48080 samples
Epoch 1/10
112070/112070 [==============================] - 3s 31us/step - loss: 0.7810 - val_loss: 0.7891
Epoch 2/10
112070/112070 [==============================] - 3s 30us/step - loss: 0.7368 - val_loss: 0.7745
Epoch 3/10
112070/112070 [==============================] - 3s 30us/step - loss: 0.7081 - val_loss: 0.7631
Epoch 4/10
112070/112070 [==============================] - 3s 30us/step - loss: 0.6856 - val_loss: 0.7537
Epoch 5/10
112070/112070 [==============================] - 3s 31us/step - loss: 0.6665 - val_loss: 0.7456
Epoch 6/10
112070/112070 [==============================] - 3s 30us/step - loss: 0.6498 - val_loss: 0.7410
Epoch 7/10
112070/112070 [==============================] - 3s 31us/step - loss: 0.6352 - val_loss: 0.7344
Epoch 8/10
112070/112070 [===========================

Epoch 10/10
112070/112070 [==============================] - 3s 30us/step - loss: 0.4286 - val_loss: 0.5326
mse = 0.10031603014873108
r2 = 0.2215217589362687
Train on 112070 samples, validate on 48080 samples
Epoch 1/10
112070/112070 [==============================] - 3s 31us/step - loss: 0.5611 - val_loss: 0.5369
Epoch 2/10
112070/112070 [==============================] - 3s 30us/step - loss: 0.5270 - val_loss: 0.5276
Epoch 3/10
112070/112070 [==============================] - 3s 30us/step - loss: 0.5059 - val_loss: 0.5206
Epoch 4/10
112070/112070 [==============================] - 3s 29us/step - loss: 0.4897 - val_loss: 0.5163
Epoch 5/10
112070/112070 [==============================] - 3s 31us/step - loss: 0.4762 - val_loss: 0.5128
Epoch 6/10
112070/112070 [==============================] - 3s 31us/step - loss: 0.4644 - val_loss: 0.5095
Epoch 7/10
112070/112070 [==============================] - 4s 31us/step - loss: 0.4545 - val_loss: 0.5066
Epoch 8/10
112070/112070 [================

112070/112070 [==============================] - 3s 31us/step - loss: 0.3563 - val_loss: 0.4230
Epoch 10/10
112070/112070 [==============================] - 3s 30us/step - loss: 0.3497 - val_loss: 0.4206
mse = 0.0959287336428779
r2 = 0.25556831023856663
Train on 112070 samples, validate on 48080 samples
Epoch 1/10
112070/112070 [==============================] - 4s 32us/step - loss: 0.4300 - val_loss: 0.4353
Epoch 2/10
112070/112070 [==============================] - 3s 31us/step - loss: 0.4008 - val_loss: 0.4259
Epoch 3/10
112070/112070 [==============================] - 3s 31us/step - loss: 0.3827 - val_loss: 0.4213
Epoch 4/10
112070/112070 [==============================] - 3s 31us/step - loss: 0.3688 - val_loss: 0.4164
Epoch 5/10
112070/112070 [==============================] - 4s 31us/step - loss: 0.3574 - val_loss: 0.4141
Epoch 6/10
112070/112070 [==============================] - 3s 30us/step - loss: 0.3476 - val_loss: 0.4134
Epoch 7/10
112070/112070 [===========================